## Carga de datos en SQL Server

In [1]:
!pip install pyodbc

In [3]:
import pyodbc

Conexión con SQL SERVER

In [5]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=Dell-Carito\SQLEXPRESS;DATABASE=datos_clima;Trusted_Connection=yes")

Carga de dataset definitivo

In [7]:
import pandas as pd

In [9]:
data = pd.read_csv(r'C:\Users\icrui\Documents\DS-ML\proy_etl_clim_arg\data\df_climas_arg.csv')

In [11]:
# Se chequea la cantidad de filas que tiene data para contrastar su version en SQL y ver si todo se cargó bien
len(data)

72

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   estacion                    72 non-null     object 
 1   mes                         72 non-null     object 
 2   provincia                   72 non-null     object 
 3   precipi_mm                  60 non-null     float64
 4   frec_dias_precipi_sup_1mm   60 non-null     float64
 5   humedad_relac               63 non-null     float64
 6   nubosidad_octas             72 non-null     float64
 7   temperat_c                  72 non-null     float64
 8   temperat_max_c              72 non-null     float64
 9   temperat_min_c              72 non-null     float64
 10  veloc_kmh_viento_2011_2020  48 non-null     float64
 11  altura_m                    72 non-null     int64  
 12  latitud                     72 non-null     float64
 13  longitud                    72 non-nu

In [17]:
cursor = conn.cursor()

Carga de datos con revisión de datos fallidos (si se presenta)

In [19]:
for index, row in data.iterrows():
    # 1) Limpieza de datos
    processed_row = row.copy()

    numeric_cols = [
        'precipi_mm', 'frec_dias_precipi_sup_1mm', 'humedad_relac', 'nubosidad_octas',
        'temperat_c', 'temperat_max_c', 'temperat_min_c', 'veloc_kmh_viento_2011_2020',
        'altura_m', 'latitud', 'longitud'
    ]

    for col in numeric_cols:
        value = processed_row[col]
        if isinstance(value, str):
            value = value.strip()
            if not value:
                processed_row[col] = None
            else:
                try:
                    processed_row[col] = float(value)
                except ValueError:
                    print(f"--- ¡ADVERTENCIA EN LA FILA {index}!: No se pudo convertir '{row[col]}' a float en la columna '{col}'. Estableciendo a None. ---")
                    processed_row[col] = None
        elif pd.isna(value):
             processed_row[col] = None

    # 2) Ejecución de la consulta con manejo de errores
    try:
        cursor.execute("""
        INSERT INTO df_climas_arg(
        estacion, mes, provincia, precipi_mm, frec_dias_precipi_sup_1mm, humedad_relac,
        nubosidad_octas, temperat_c, temperat_max_c, temperat_min_c, veloc_kmh_viento_2011_2020,
        altura_m, latitud, longitud) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        processed_row['estacion'], processed_row['mes'], processed_row['provincia'], processed_row['precipi_mm'],
        processed_row['frec_dias_precipi_sup_1mm'], processed_row['humedad_relac'], processed_row['nubosidad_octas'], processed_row['temperat_c'],
        processed_row['temperat_max_c'], processed_row['temperat_min_c'], processed_row['veloc_kmh_viento_2011_2020'],
        processed_row['altura_m'], processed_row['latitud'], processed_row['longitud'])

    except pyodbc.ProgrammingError as e:
        print(f"\n--- ¡ERROR DE INSERCIÓN DETECTADO EN LA FILA {index}! ---")
        print(f"Error completo: {e}")
        print(f"Diccionario de datos de la fila problemática: {processed_row.to_dict()}")

        print("\n--- DETALLE DE PARÁMETROS ENVIADOS A LA BASE DE DATOS PARA ESTA FILA ---")
        params = [
            processed_row['estacion'], processed_row['mes'], processed_row['provincia'], processed_row['precipi_mm'],
            processed_row['frec_dias_precipi_sup_1mm'], processed_row['humedad_relac'], processed_row['nubosidad_octas'], processed_row['temperat_c'],
            processed_row['temperat_max_c'], processed_row['temperat_min_c'], processed_row['veloc_kmh_viento_2011_2020'],
            processed_row['altura_m'], processed_row['latitud'], processed_row['longitud']
        ]
        for i, p_val in enumerate(params):
            prefix = f"Parámetro {i+1}"
            if i == 13: # El parámetro 14 es el índice 13
                prefix = "**Parámetro 14 (lon_gr)**"
            print(f"{prefix}: {p_val} (Tipo Python: {type(p_val)})")

        conn.rollback()
        cursor.close()
        conn.close()
        raise

    except Exception as e:
        print(f"\n--- ¡ERROR INESPERADO EN LA FILA {index}! ---")
        print(f"Error: {e}")
        print(f"Diccionario de datos de la fila problemática: {processed_row.to_dict()}")
        conn.rollback()
        cursor.close()
        conn.close()
        raise

conn.commit()
cursor.close()
conn.close()